In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Jul 22 18:27:31 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
!pip install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.7 MB/s eta 0:00:00


In [5]:
!pip install transformers -q

In [6]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd

import string
import re
import nltk
from nltk.corpus import stopwords

from google.colab import drive
drive.mount('/drive')

from torch.optim import Adam
import torch
from torch.utils.data import DataLoader

Mounted at /drive


In [7]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>",
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})

tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_tokens(["<response>:"])

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

Embedding(50261, 768)

In [8]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

model = model.to(device)

In [9]:
train = pd.read_csv('/drive/My Drive/Colab Notebooks/dissertation/train.csv')
test = pd.read_csv('/drive/My Drive/Colab Notebooks/dissertation/test.csv')

In [10]:
train['input'] = "<startofstring> "+ train['Questions'] +" <response>: "+ train['Answers'] +" <endofstring>"
text_input = list(train['input'])

In [11]:
from torch.utils.data import Dataset

class ChatData(Dataset):
    def __init__(self, data_in:list, tokenizer):
        self.data = data_in

        # Set pad_token if not set
        if tokenizer.pad_token is None:
          tokenizer.pad_token = tokenizer.eos_token

        self.X_encoded = tokenizer(self.data,max_length=40, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [12]:
chatData = ChatData(text_input, tokenizer)
chatData =  DataLoader(chatData, batch_size=8)

In [13]:
def train_model(chatData,model,optim):
  epochs = 10

  for i in range(epochs):
    for x,a in chatData:
      x = x.to(device)
      a = a.to(device)
      optim.zero_grad()
      loss = model(
          x,
          attention_mask = a,
          labels = x
      ).loss
      loss.backward()
      optim.step()
    torch.save(model.state_dict(),"model_state.pt")
    print(infer("How do I communicate data-driven insights to stakeholders within my startup?"))

In [14]:
def infer(inp):
  inp = "<startofstring> "+inp+" <response>: "
  inp = tokenizer(inp, return_tensors="pt")
  X = inp["input_ids"].to(device)
  a = inp["attention_mask"].to(device)
  output = model.generate(X, attention_mask=a,max_length=80)
  output = tokenizer.decode(output[0], skip_special_tokens=True)
  return output

In [15]:
model.train()
optim = Adam(model.parameters())
train_model(chatData,model,optim)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How do I communicate data-driven insights to stakeholders within my startup? <response>: Encourage representatives to share their insights and insights.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How do I communicate data-driven insights to stakeholders within my startup? <response>: Clearly communicate the insights of the insights and insights.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

How do I communicate data-driven insights to stakeholders within my startup? <response>: Clearly articulate the implications and recommendations based on the insights.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How do I communicate data-driven insights to stakeholders within my startup? <response>: Clearly communicate the implications and recommendations for the different channels.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


How do I communicate data-driven insights to stakeholders within my startup? <response>: Clearly articulate the implications and implications of the business.


In [16]:
test['gpt2'] = test.apply(lambda x: infer(x['Questions']), axis = 1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [17]:
test.to_csv('/drive/My Drive/Colab Notebooks/dissertation/test.csv')